In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings

import sys
from pyprojroot import here
sys.path.append("../..") 
# from laos_gggi.statistics import get_distance_to_rivers

import pandas as pd
import numpy as np
import pymc as pm
import pytensor
import pytensor.tensor as pt
import matplotlib.pyplot as plt
import arviz as az
import geopandas as gpd
import xarray as xr

from laos_gggi.replication_data import create_replication_data
from laos_gggi.plotting import plotting_function_damages 
from laos_gggi.model import add_data
from laos_gggi.plotting import configure_plot_style

from laos_gggi.sample import sample_or_load
from sklearn.preprocessing import StandardScaler as Standardize

from laos_gggi.data_functions import load_emdat_data, load_shapefile, load_rivers_data

In [3]:
# Set random seed
SEED = sum(list(map(ord, 'climate_bayes')))
rng = np.random.default_rng(SEED)

# Loading and preparing data 

In [4]:
# Select SEA countries
laos_neighboors = [
    "KHM",  # Cambodia
    "THA",  # Thailand
    "LAO",  # Laos
    "VNM",  # Vietnam
]

In [5]:
# Configure pytensir floats
floatX = pytensor.config.floatX


# Load idatas
event_idata = az.from_netcdf("model_closest_full_long.idata")
damage_idata = az.from_netcdf("damages_model.idata")

#Rename ISO dimension to avoid conflicts merging
damage_idata =damage_idata.rename({"ISO": "ISO_damage",
                                                   "country_effect": "country_effect_damage"})

#Merge posteriors
merged_posteriors = xr.merge([event_idata.sel(ISO = laos_neighboors).posterior,
                              damage_idata.posterior])

In [6]:
# Load maps
world = load_shapefile('world')

# Define maps
sea_map = world.query('ISO_A3 in @laos_neighboors')
laos_map = world.query('ISO_A3 == "LAO"')

### Events data

In [7]:
#Load data sets
sea_point_grid = pd.read_csv(here("data/sea_point_grid.csv"),index_col=0  ).rename(columns = {"ISO_A3": "ISO"})
laos_point_grid = pd.read_csv(here("data/laos_point_grid.csv"),index_col=0 ).rename(columns = {"ISO_A3": "ISO"})
sea_df = pd.read_csv(here("data/sea.csv"),index_col=0 )
lao_df = pd.read_csv(here("data/lao.csv"),index_col=0 )
sea_df_stand = pd.read_csv(here("data/sea_df_stand.csv"),index_col=0 )
lao_df_stand = pd.read_csv(here("data/lao_df_stand.csv"),index_col=0 )

predictions = pd.read_csv(here("data/climate_forecast.csv")).rename(columns = {"time": "Start_Year"})

In [8]:
# create predictions_ISO
predictions["ISO"] = "LAO"

# Select years of interest
prediction_years = ["2026-01-01", "2030-01-01", "2040-01-01", "2050-01-01", "2070-01-01"]
predictions_short = predictions.query('Start_Year in @prediction_years')

In [9]:
# create grid point number
laos_point_grid = laos_point_grid.reset_index().rename(columns = {"index": "point_number" })

# merge grid and predictions
pred_df = pd.merge(laos_point_grid ,predictions_short, left_on= "ISO", right_on= "ISO", how = "left" )

In [10]:
# Create population density
pred_df["population_density"] = pred_df["Population"] / 236800 # Laos area in squared km hardcoded

# Calculate logs
pred_df["log_population_density"] = np.log(pred_df["population_density"])
pred_df["log_gdp_per_cap"] = np.log(pred_df["gdp_per_cap"])

In [11]:
# Define features
event_features = ['lat', 'long', 'log_distance_to_river__standardized', "log_distance_to_coastline__standardized",
"log_distance_to_river__standardized_squared", "log_distance_to_coastline__standardized_squared",
 "Population__standardized", "co2__standardized", "precip_deviation__standardized", "dev_ocean_temp__standardized", 
'log_population_density__standardized','log_gdp_per_cap__standardized', 
"log_gdp_per_cap__standardized_squared", "log_population_density__standardized_squared" ]

distance_features = event_features[0:6]
time_varying_features = event_features[6:]
time_varying_features_base =[x[ :-14] for x in event_features[6:12]]

cols_to_stand = ['Population','log_distance_to_river', 'dev_ocean_temp',
                 'co2', 'log_population_density', 'log_gdp_per_cap',
       'precip_deviation', 'log_distance_to_coastline',]

cols_to_stand_stand = [x + "__standardized" for x in  cols_to_stand]

cols_not_stand = ['ISO' , 'Start_Year', 'lat', 'long', 'geometry', 'point_number']

In [12]:
#Train transformers
transformer_stand_ =  Standardize().fit(sea_df.query('is_disaster == 1')[cols_to_stand] )

In [13]:
# standardize pred_df
pred_df_stand = transformer_stand_.transform(pred_df[cols_to_stand] )
pred_df_stand = pd.DataFrame(pred_df_stand, columns =cols_to_stand_stand )
pred_df_stand = pd.merge(pred_df_stand, pred_df[cols_not_stand], left_index=True, right_index= True, how = "left" )

In [14]:
# Calculate logs squared
pred_df_stand["log_gdp_per_cap__standardized_squared"] = pred_df_stand["log_gdp_per_cap__standardized"] **2
pred_df_stand["log_population_density__standardized_squared"] = pred_df_stand["log_population_density__standardized"] **2

pred_df_stand["log_distance_to_river__standardized_squared"] = pred_df_stand["log_distance_to_river__standardized"] **2
pred_df_stand["log_distance_to_coastline__standardized_squared"] = pred_df_stand["log_distance_to_coastline__standardized"] **2


### Center Laos lat-long using SEA data

In [15]:
# Compute sea center
sea_center =  {}

# center Laos values
for x in ["lat", "long"]:
    sea_center[x] = (sea_df[x].max() + sea_df[x].min()) /  2
    pred_df_stand[x + "_centered"] = pred_df_stand[x] - sea_center[x]

In [16]:
### Create pred_df_stand_dict for several years
pred_df_stand_dict = {}

for year in pred_df_stand["Start_Year"].unique():
    pred_df_stand_dict[year] = pred_df_stand.query('Start_Year == @year')

# Set coords

In [17]:
# Load damage df
damage_df_stand = pd.read_csv(here("data/damage_df_stand.csv"),index_col =0 )

In [18]:
# gp features
gp_features = ["lat", "long"]

# damage features
damage_features = ['Population__standardized',
 'log_population_density__standardized',
 'log_population_density__standardized_squared',
 'log_gdp_per_cap__standardized',
 'log_gdp_per_cap__standardized_squared',
 'dev_ocean_temp__standardized',
 'co2__standardized',
 'precip_deviation__standardized']

# Event features
event_features = ['log_distance_to_river__standardized',
 'log_distance_to_coastline__standardized',
 'Population__standardized',
 'co2__standardized',
 'precip_deviation__standardized',
 'dev_ocean_temp__standardized',
 'log_population_density__standardized',
 'log_gdp_per_cap__standardized',
 'log_gdp_per_cap__standardized_squared',
 'log_population_density__standardized_squared',
 'log_distance_to_river__standardized_squared',
 'log_distance_to_coastline__standardized_squared']

In [19]:
# Define event coords
is_disaster_idx_events , is_disaster_events  = pd.factorize(sea_df_stand["is_disaster"])
ISO_idx_events , ISO_events  = pd.factorize(sea_df_stand["ISO"]) 
obs_idx_events  = sea_df_stand.index

#Creating idx
xr_idx_events  = xr.Coordinates.from_pandas_multiindex(sea_df_stand.set_index(['ISO', 'Start_Year']).index, 'obs_idx')

#Set coords
event_coords = {"is_disaster" : is_disaster_events,
        "obs_idx": obs_idx_events,
        "ISO": ISO_events,
        "feature": event_features,
        "gp_feature":gp_features }


In [20]:
# Define damage coords
ISO_idx_damage, ISO_damage =  pd.factorize(damage_df_stand["ISO"])
coords_damage = {
    'ISO':ISO_damage,
    'obs_idx':damage_df_stand.index,
    'feature': damage_features
        }    

xr_idx_damage = xr.Coordinates.from_pandas_multiindex(damage_df_stand.set_index(['ISO', 'year']).index, 'obs_idx')


In [21]:
# Prediction coords
obs_idx = pred_df_stand_dict["2026-01-01"]["point_number"]


gp_features = ["lat", "long"]


# Rebuild ISO_idx_sea
ISO_idx, ISO = pd.factorize(sea_df["ISO"]) 

ISO_to_idx = {name: idx for idx, name in enumerate(ISO)}
ISO_idx_laos =  pred_df_stand_dict["2026-01-01"].ISO.map(ISO_to_idx.get)

years = pred_df_stand["Start_Year"].unique()

# ISO damage
ISO_damage_idx, ISO_damage =  pd.factorize(damage_df_stand["ISO"])
ISO_to_idx_2 = {name: idx for idx, name in enumerate(ISO_damage)}
ISO_damage_idx_laos =  pred_df_stand_dict["2026-01-01"].ISO.map(ISO_to_idx_2.get)

# 


#Set coords_predictions
coords_predictions = {
    # "is_disaster" : is_disaster,
        "obs_idx": obs_idx,
        "ISO": ISO,
        "distance_features": distance_features,
        "time_varying_features" : time_varying_features,
        "gp_feature":gp_features,
        "ISO_damage": ISO_damage ,
        "damage_features": damage_features,
        "event_features": event_features,
        "year":years
                     }

# Compute the curves

In [22]:
# Compute center function
def compute_center(X):
    return (pt.max(X, axis=0) + pt.min(X, axis=0)).eval() / 2

In [23]:
from copy import deepcopy
from pymc.model.transform.optimization import freeze_dims_and_data

damage_curves_plot = {}

for year in years:
    
    with pm.Model(coords=coords_predictions) as damage_curves_plot[year]:
        ####################################Events model####################################
        # Set data
        event_features_data = add_data(features= event_features ,  target = None, df =  pred_df_stand_dict[year],
                                   dims=['obs_idx', 'features'])
        X_gp = pm.Data("X_gp",  pred_df_stand_dict[year][["lat_centered", "long_centered"]].astype(floatX), dims=['obs_idx', 'gp_feature'])
    
        #Flat variables
        country_effect = pm.Flat("country_effect", dims = ["ISO"])    
        beta = pm.Flat("beta", dims = ["event_features"])
        
        # HSGP component
        eta = pm.Flat("eta")
        ell = pm.Flat("ell", dims=["gp_feature"])
        cov_func = eta**2 * pm.gp.cov.Matern52(input_dim=2, ls=ell)
    
        m0, m1, c = 35, 35, 1.5
        gp = pm.gp.HSGP(m=[m0, m1], c=c, cov_func=cov_func)
        gp._X_center = compute_center( pred_df_stand_dict[year][["lat_centered", "long_centered"]].values.astype(floatX))
    
        phi, sqrt_psd = gp.prior_linearized(X=X_gp)
    
        basis_coeffs = pm.Flat("basis_coeffs", size=gp.n_basis_vectors)
    
        HSGP_component = pm.Deterministic('HSGP_component', phi @ (basis_coeffs * sqrt_psd), dims=['obs_idx'])
    
        # Event model components
        event_features_component = pm.Deterministic('event_features_component', 
                                                    event_features_data @ beta, dims=['obs_idx'])
        
        logit_p = pm.Deterministic('logit_p', country_effect[ISO_idx_laos] + event_features_component 
                                   + HSGP_component, dims=['obs_idx'])
        event_prob_y_hat = pm.Deterministic('event_prob_y_hat', pm.math.invlogit(logit_p), dims=['obs_idx'])
    
        ####################################Damages model####################################
        # Set data
        damage_x_data = pm.Data("damage_x_data",   pred_df_stand_dict[year][damage_features], dims=['obs_idx', 'damage_features'])
    
        # Set flats
        country_effect_damage = pm.Flat("country_effect_damage", dims = ["ISO_damage"])
        betas_damage = pm.Flat("betas_damage", dims = ["damage_features"])
        sigma_damage = pm.Flat("sigma_damage")
    
        #Damage model components
        mu =  country_effect_damage[ISO_damage_idx_laos] + damage_x_data @ betas_damage
       
        ln_damage_millions = pm.Normal("ln_damage_millions", mu = mu, sigma = sigma_damage , dims = ["obs_idx"])
    
        damage_millions = pm.Deterministic("damage_millions", pm.math.exp(ln_damage_millions), dims=['obs_idx'])
    
        # #################################### Damage curves ####################################
        damages_curves = pm.Deterministic("damages_curves", damage_millions * event_prob_y_hat, dims=['obs_idx'])

In [24]:
# Sample predictions
idata_plot_point = {}
for year in years:
    with freeze_dims_and_data(damage_curves_plot[year] ):
        idata_plot_point[year] = pm.sample_posterior_predictive(merged_posteriors , extend_inferencedata=False, 
                                                          compile_kwargs={'mode':'JAX'},
                                                    var_names=[
                                                               'HSGP_component', 
                                                               'event_features_component', 
                                                               'logit_p', 
                                                               'event_prob_y_hat', 
                                                               "ln_damage_millions",
                                                                "damage_millions", 
                                                               "damages_curves"
                                                              ])

C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pytensor\link\jax\linker.py:27: UserWarning: The RandomType SharedVariables [RNG(<Generator(PCG64) at 0x2353FC0B140>)] will not be used in the compiled JAX graph. Instead a copy will be used.
  warnings.warn(
Sampling: [ln_damage_millions]


Output()

C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pytensor\link\jax\linker.py:27: UserWarning: The RandomType SharedVariables [RNG(<Generator(PCG64) at 0x235777EB840>)] will not be used in the compiled JAX graph. Instead a copy will be used.
  warnings.warn(
Sampling: [ln_damage_millions]


Output()

C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pytensor\link\jax\linker.py:27: UserWarning: The RandomType SharedVariables [RNG(<Generator(PCG64) at 0x2353F85EEA0>)] will not be used in the compiled JAX graph. Instead a copy will be used.
  warnings.warn(
Sampling: [ln_damage_millions]


Output()

C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pytensor\link\jax\linker.py:27: UserWarning: The RandomType SharedVariables [RNG(<Generator(PCG64) at 0x235476B97E0>)] will not be used in the compiled JAX graph. Instead a copy will be used.
  warnings.warn(
Sampling: [ln_damage_millions]


Output()

C:\Users\camil\miniforge3\envs\laos-climate-change\Lib\site-packages\pytensor\link\jax\linker.py:27: UserWarning: The RandomType SharedVariables [RNG(<Generator(PCG64) at 0x2353FF9D540>)] will not be used in the compiled JAX graph. Instead a copy will be used.
  warnings.warn(
Sampling: [ln_damage_millions]


Output()

In [52]:
idata_plot_point["2026-01-01"].posterior_predictive["HSGP_component"]

<xarray.DataArray 'HSGP_component' (chain: 8, draw: 1000, obs_idx: 11901)> Size: 762MB
array([[[-3.27444314, -3.27947647, -3.28368498, ..., -2.98479162,
         -3.00359736, -3.02237199],
        [-3.71141193, -3.70920643, -3.70614578, ..., -3.72805703,
         -3.73583204, -3.74307974],
        [-2.98254213, -2.94556645, -2.90894357, ..., -3.93035115,
         -3.96662905, -4.00120343],
        ...,
        [-3.48840366, -3.45889916, -3.42798821, ..., -2.82886749,
         -2.84753679, -2.86593357],
        [-2.18999389, -2.17980942, -2.17059341, ..., -2.7770144 ,
         -2.7981141 , -2.81919777],
        [-1.98955154, -1.97519278, -1.96084776, ..., -2.15843562,
         -2.17771729, -2.19732421]],

       [[-2.11685755, -2.11106234, -2.10535409, ..., -2.35036128,
         -2.35743153, -2.36478958],
        [-2.07128902, -2.05583019, -2.04054828, ..., -2.26119649,
         -2.2707759 , -2.28061631],
        [-1.84464224, -1.82388682, -1.80352199, ..., -2.42095019,
         -2.43294524, -2.44463693],
...
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]]])
Coordinates:
  * chain    (chain) int32 32B 0 1 2 3 4 5 6 7
  * draw     (draw) int32 4kB 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * obs_idx  (obs_idx) int32 48kB 0 1 2 3 4 5 ... 11896 11897 11898 11899 11900

In [36]:
for year in years:
    print(pred_df_stand_dict[year].shape())

TypeError: 'tuple' object is not callable